<a href="https://colab.research.google.com/github/allnes/pcam_train/blob/master/pcam_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd  '/content/drive/My Drive/DL_PCAM'

/content/drive/My Drive/DL_PCAM


In [4]:
%ls

camelyonpatch_level_2_split_test_meta.csv
camelyonpatch_level_2_split_test_x.h5
camelyonpatch_level_2_split_test_x.h5.gz
camelyonpatch_level_2_split_test_y.h5
camelyonpatch_level_2_split_test_y.h5.gz
camelyonpatch_level_2_split_train_meta.csv
camelyonpatch_level_2_split_train_x.h5
camelyonpatch_level_2_split_train_x.h5.gz
camelyonpatch_level_2_split_train_y.h5
camelyonpatch_level_2_split_train_y.h5.gz
camelyonpatch_level_2_split_valid_meta.csv
camelyonpatch_level_2_split_valid_x.h5
camelyonpatch_level_2_split_valid_x.h5.gz
camelyonpatch_level_2_split_valid_y.h5
camelyonpatch_level_2_split_valid_y.h5.gz
__pycache__/


In [0]:
import gzip
import shutil
def uzip_file(file_name, origin_name):
  with gzip.open(file_name, 'rb') as f_in:
      with open(origin_name, 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)


In [0]:
train_x_name = 'camelyonpatch_level_2_split_train_x.h5'
train_y_name = 'camelyonpatch_level_2_split_train_y.h5'

test_x_name = 'camelyonpatch_level_2_split_test_x.h5'
test_y_name = 'camelyonpatch_level_2_split_test_y.h5'

valid_x_name = 'camelyonpatch_level_2_split_valid_x.h5'
valid_y_name = 'camelyonpatch_level_2_split_valid_y.h5'

meta_train_name = 'camelyonpatch_level_2_split_train_meta.csv'
meta_valid_name = 'camelyonpatch_level_2_split_valid_meta.csv'
meta_test_name  = 'camelyonpatch_level_2_split_test_meta.csv'

# uzip_file(train_x_name + '.gz', train_x_name)
# uzip_file(train_y_name + '.gz', train_y_name)

# uzip_file(test_x_name + '.gz', test_x_name)
# uzip_file(test_y_name + '.gz', test_y_name)

# uzip_file(valid_x_name + '.gz', valid_x_name)
# uzip_file(valid_y_name + '.gz', valid_y_name)


In [7]:
from keras.utils import HDF5Matrix
import pandas as pd

x_train = HDF5Matrix(train_x_name, 'x', start=10000, end=20000)
y_train = HDF5Matrix(train_y_name, 'y', start=10000, end=20000)

x_valid = HDF5Matrix(valid_x_name, 'x')
y_valid = HDF5Matrix(valid_y_name, 'y')

x_test = HDF5Matrix(test_x_name, 'x')
y_test = HDF5Matrix(test_y_name, 'y')

meta_train = pd.read_csv(meta_train_name)
meta_valid = pd.read_csv(meta_valid_name)
meta_test = pd.read_csv(meta_test_name)

Using TensorFlow backend.


In [8]:
print(len(x_train))
print(type(x_train))
print(type(x_train[0]))
print(x_train[0].shape)


262144
<class 'keras.utils.io_utils.HDF5Matrix'>
<class 'numpy.ndarray'>
(96, 96, 3)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Lambda, Input
from keras.regularizers import l2
import tensorflow as tf

In [0]:
full_size = 32 * 32 * 3
original_shape = (96, 96, 3)
new_shape = (32, 32)

In [43]:
model = Sequential([
    # Input(original_shape),
    Lambda(lambda image: tf.image.resize(image, new_shape)),
    Reshape((1, 1, full_size,), input_shape=x_train[0].shape),
    Dense(10000), Activation('relu'),
    Dense(5000),  Activation('relu'),
    Dense(1),     Activation('softmax'),
])
model.build((len(x_train), 96, 96, 3))
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_7 (Lambda)            (262144, 32, 32, 3)       0         
_________________________________________________________________
reshape_17 (Reshape)         (262144, 1, 1, 3072)      0         
_________________________________________________________________
dense_55 (Dense)             (262144, 1, 1, 10000)     30730000  
_________________________________________________________________
activation_55 (Activation)   (262144, 1, 1, 10000)     0         
_________________________________________________________________
dense_56 (Dense)             (262144, 1, 1, 5000)      50005000  
_________________________________________________________________
activation_56 (Activation)   (262144, 1, 1, 5000)      0         
_________________________________________________________________
dense_57 (Dense)             (262144, 1, 1, 1)       

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [45]:
model.fit(x_train, y_train, epochs=10, batch_size=16384, shuffle='batch')




Epoch 1/10







ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-45-24f86f1e7076>", line 1, in <module>
    model.fit(x_train, y_train, epochs=10, batch_size=16384, shuffle='batch')
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1178, in fit
    validation_freq=validation_freq)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_arrays.py", line 204, in fit_loop
    outs = fit_function(ins_batch)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2959, in __call__
    if hasattr(get_session(), '_make_callable_from_options'):
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 223, in get_session
    session.run(tf.variables_initializer(uninitialized_vars))
  File "/usr/local/lib/python3.6/dist-packages/tensorflow

KeyboardInterrupt: ignored

In [0]:
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy * 100))